# One click ComfyUI ⚡

*if you find this helpful consider becoming a member on patreon, subscribe to my youtube for Ai applications guides*

[![Patreon](https://img.shields.io/badge/Patreon-Support-orange?style=rounded-square&logo=patreon)](https://www.patreon.com/intuitiveai)
[![Youtube](https://img.shields.io/badge/Youtube-Subscribe-red?style=rounded-square&logo=youtube)](https://www.youtube.com/@theintuitiveye)
[![GitHub](https://img.shields.io/badge/GitHub-Visit-blue?style=rounded-square&logo=github)](https://github.com/tieai)
[![Huggingface](https://img.shields.io/badge/Huggingface-visit-yellow?style=rounded-square&logo=huggingface)](https://huggingface.co/theintuitiveye)
[![Instagram](https://img.shields.io/badge/Instagarm-Follow-pink?style=rounded-square&logo=Instagram)](https://Instagram.com/theintuitiveye)



In [ ]:
# @title One Click Setup
import os
!apt -y update -qq
!apt -y install -qq aria2
!pip install -q xformers==0.0.20

!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -d /content/ComfyUI/models/upscale_models -o RealESRGAN_x2plus.pth

base = "https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base} -d /content/ComfyUI/models/checkpoints -o sd_xl_base_1.0.safetensors

refiner = "https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {refiner} -d /content/ComfyUI/models/checkpoints -o sd_xl_refiner_1.0.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors -d /content/ComfyUI/models/vae -o sdxl_vae.vae.safetensors

!python main.py --dont-print-server

In [ ]:
# @title Save outputs to drive - Run it once before disconnecting the runtime { display-mode: "both" }
from google.colab import drive
drive.mount('/content/drive')

import shutil
import os
import datetime
# Get the user's desired folder name
output_folder_name="Enter folder name here" #@param{type:"string"}
# Define paths
source_folder_path = '/content/ComfyUI/output'  # Replace with the actual path to the folder in the runtime environment
destination_folder_path = f'/content/drive/MyDrive/{output_folder_name}'  # Replace with the desired destination path in your Google Drive

# Create the destination folder in Google Drive if it doesn't exist
os.makedirs(destination_folder_path, exist_ok=True)

# Get the current date in DD/MM/YY format
current_date = datetime.datetime.now().strftime("%d-%m-%y")
date_folder_path = os.path.join(destination_folder_path, current_date)

# Create the date folder in the destination folder
os.makedirs(date_folder_path, exist_ok=True)

# Copy the contents of the source folder to the date folder
for item_name in os.listdir(source_folder_path):
    item_path = os.path.join(source_folder_path, item_name)
    destination_item_path = os.path.join(date_folder_path, item_name)

    # Copy the item to the date folder
    shutil.copy(item_path, destination_item_path)

print("Contents copied successfully to the date folder:", current_date)

